# Run mAP line-by-line
**Author:** Jessica Ewald <br>

The purpose of this script is to run through mAP line-by-line for a few cells that return a mAP of 1 to try understand why this happens. Code chunks will be copied from the copairs repo to accomplish this. 

In [1]:
# general imports
import pathlib
import pandas as pd
import numpy as np
import polars as pl
import seaborn as sns
import black
import jupyter_black
jupyter_black.load(
    lab=False,
    line_length=79,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)
import warnings
warnings.filterwarnings("ignore")

# copairs imports
import itertools
import copairs as cps
from copairs.matching import Matcher
from copairs import compute
from typing import List, Tuple


DEBUG:jupyter_black:config: {'line_length': 79, 'target_versions': {<TargetVersion.PY310: 10>}}


<IPython.core.display.Javascript object>

Function defined by me:

In [3]:
# Function for formatting map input
def prep_for_map(df_path: str, map_cols: [str], sample_col: [str], sample_n: int = 5): # type: ignore

    # define filters
    q = pl.scan_parquet(df_path).filter(
        (pl.col("Metadata_node_type") != "TC") &  # remove transfection controls
        (pl.col("Metadata_node_type") != "NC") &
        (pl.col("Metadata_node_type") != "PC") &
        (pl.col("Metadata_node_type") != "CC") &
        (pl.col("Metadata_allele") != "_NA") & 
        (pl.sum_horizontal(pl.col(map_cols).is_null()) == 0)  # remove any row with missing values for selected meta columns
        ).with_columns(pl.concat_str(sample_col).alias('Metadata_samplecol'))
    
    # if a sample column name was provided, randomly sample sample_n rows from each column category
    if sample_col:
        q = q.filter(pl.int_range(0, pl.len()).shuffle().over('Metadata_samplecol') < sample_n)
    
    # different data frames for metadata and profiling data
    map_cols_id = map_cols.copy()
    # map_cols_id.append("Metadata_CellID")
    meta_cols = q.select(map_cols_id)
    meta_df = meta_cols.collect().to_pandas()

    feat_col = [i for i in q.columns if "Metadata_" not in i] 
    q = q.select(feat_col)
    feat_df = q.collect().to_pandas()

    map_input = {'meta': meta_df, 'feats': feat_df}

    return map_input


Copies of copairs functions that are not exported.

In [ ]:
def flatten_str_list(*args):
    """create a single list with all the params given"""
    columns = set()
    for col in args:
        if isinstance(col, str):
            columns.add(col)
        elif isinstance(col, dict):
            columns.update(itertools.chain.from_iterable(col.values()))
        else:
            columns.update(col)
    columns = list(columns)
    return columns

def evaluate_and_filter(df, columns) -> Tuple[pd.DataFrame, List[str]]:
    """Evaluate the query and filter the dataframe"""
    parsed_cols = []
    for col in columns:
        if col in df.columns:
            parsed_cols.append(col)
            continue

        column_names = re.findall(r"(\w+)\s*[=<>!]+", col)
        valid_column_names = [col for col in column_names if col in df.columns]
        if not valid_column_names:
            raise ValueError(f"Invalid query or column name: {col}")

        try:
            df = df.query(col)
            parsed_cols.extend(valid_column_names)
        except:
            raise ValueError(f"Invalid query expression: {col}")

    return df, parsed_cols

def build_rank_lists(pos_pairs, neg_pairs, pos_sims, neg_sims):
    labels = np.concatenate(
        [
            np.ones(pos_pairs.size, dtype=np.int32),
            np.zeros(neg_pairs.size, dtype=np.int32),
        ]
    )
    ix = np.concatenate([pos_pairs.ravel(), neg_pairs.ravel()])
    sim_all = np.concatenate([np.repeat(pos_sims, 2), np.repeat(neg_sims, 2)])
    ix_sort = np.lexsort([1 - sim_all, ix])
    rel_k_list = labels[ix_sort]
    paired_ix, counts = np.unique(ix, return_counts=True)
    return paired_ix, rel_k_list, counts

Define some paths / average precision function inputs

In [4]:
# Set paths for accessing data
batch_name = 'B1A1R1'
data_dir = pathlib.Path("/dgx1nas1/storage/data/jess/varchamp/sc_data/processed_profiles").resolve(strict=True)
anno_cellID = pathlib.Path(data_dir / f"{batch_name}_annotated.parquet")

# Set paramters for mAP
pos_sameby = ['Metadata_allele']
pos_diffby = ['Metadata_Plate']
neg_sameby = ['Metadata_Plate']
neg_diffby = ['Metadata_allele']
batch_size = 20000
sample_n_cells = 5
sample_neg = True
map_cols = list(set(pos_sameby + pos_diffby + neg_sameby + neg_diffby))


Reformat data for copairs. This step took ~4.5 minutes when DGX was empty.

In [5]:
# Prepare the data (filter, sample, & format)
map_input = prep_for_map(anno_cellID, map_cols, ['Metadata_Well', 'Metadata_Plate'], sample_n_cells)

In [6]:
# Define map inputs
meta = map_input['meta']
feats = map_input['feats'].values
sample_factor = 10

The "average_precision" function starts here. It is broken down into chunks. Some extra plots etc are added to further investigate the results. 

In [7]:
# Format inputs & define matcher
columns = flatten_str_list(pos_sameby, pos_diffby, neg_sameby, neg_diffby)
meta = meta.reset_index(drop=True).copy()

matcher = Matcher(*evaluate_and_filter(meta, columns), seed=0)


In [8]:
# Compute positive pair indices
pos_pairs = matcher.get_all_pairs(sameby=pos_sameby, diffby=pos_diffby)
pos_total = sum(len(p) for p in pos_pairs.values())

pos_pairs = np.fromiter(
    itertools.chain.from_iterable(pos_pairs.values()),
    dtype=np.dtype((np.int32, 2)),
    count=pos_total,
)

In [9]:
# Compute negative pair indices
neg_pairs = matcher.get_all_pairs(sameby=neg_sameby, diffby=neg_diffby)
neg_total = sum(len(p) for p in neg_pairs.values())
neg_pairs = np.fromiter(
    itertools.chain.from_iterable(neg_pairs.values()),
    dtype=np.dtype((np.int32, 2)),
    count=neg_total,
)

In [10]:
# if sample_neg, randomly sample negative pairs
# in this case it reduces # pairs from ~21 million to ~2 million
if sample_neg:
    sample_size = pos_pairs.shape[0]*sample_factor
    if sample_size < neg_pairs.shape[0]:
        sampled_rows = np.random.choice(neg_pairs.shape[0], size=sample_size, replace=False)
        neg_pairs = neg_pairs[sampled_rows]

In [36]:
# Compute positive cosine distances
pos_sims = compute.pairwise_cosine(feats, pos_pairs, batch_size)
print(f'There are {np.round((sum(np.isnan(pos_sims))/len(pos_sims)) *100)}% NaN pairwise similarity values in the positive pairs')

  0%|          | 0/11 [00:00<?, ?it/s]

There are 25.0% NaN pairwise similarity values in the positive pairs


In [20]:
# Compute negative cosine distances
neg_sims = compute.pairwise_cosine(feats, neg_pairs, batch_size)
print(f'There are {np.round((sum(np.isnan(neg_sims))/len(neg_sims)) *100)}% NaN pairwise similarity values in the negative pairs')

  0%|          | 0/102 [00:00<?, ?it/s]

There are 25.0% NaN pairwise similarity values in the negative pairs


In [21]:
# Build ranked lists
paired_ix, rel_k_list, counts = build_rank_lists(
    pos_pairs, neg_pairs, pos_sims, neg_sims
)
# Compute average precision
ap_scores, null_confs = compute.ap_contiguous(rel_k_list, counts)

In [22]:
# Populate metadata with results
meta["n_pos_pairs"] = 0
meta["n_total_pairs"] = 0
meta.loc[paired_ix, "average_precision"] = ap_scores
meta.loc[paired_ix, "n_pos_pairs"] = null_confs[:, 0]
meta.loc[paired_ix, "n_total_pairs"] = null_confs[:, 1]

In [37]:
# put sims and pairs side-by-side
pos_sims = pos_sims.reshape(-1, 1) 
pos_res = np.concatenate((pos_pairs, pos_sims), axis = 1)

# add the reverse version so that all pairs corresponding to one cell ID can be accessed through searching one column
pos_res = np.concatenate((pos_res, pos_res[:, [1,0,2]]), axis = 0)


Here is an example of a profile where the pairwise similarity values are NaN for every pair. 

In [43]:
# show that for an example profile with a NaN, all pairwise similarities are NaNs
pos_res = pd.DataFrame(pos_res, columns = ['First_profile', 'Second_profile', 'Pw_sim'])
pos_res.loc[pos_res['First_profile'] == 14322]


,First_profile,Second_profile,Pw_sim
149,14322.0,6519.0,NaN
202862,14322.0,6541.0,NaN
202877,14322.0,6542.0,NaN
202892,14322.0,6543.0,NaN
202907,14322.0,6546.0,NaN
202918,14322.0,21316.0,NaN
202929,14322.0,21317.0,NaN
202940,14322.0,21318.0,NaN
202951,14322.0,21319.0,NaN
202969,14322.0,21339.0,NaN


In [45]:
# For other profiles, the pairwise sim is only NaN when the other profile has a NaN
pos_res.loc[pos_res['First_profile'] == 6542]

,First_profile,Second_profile,Pw_sim
15,6542.0,21316.0,0.976342
16,6542.0,21317.0,0.955220
17,6542.0,21318.0,0.963460
18,6542.0,21319.0,0.912039
19,6542.0,19556.0,NaN
20,6542.0,19557.0,0.966662
21,6542.0,19558.0,0.775481
22,6542.0,19559.0,NaN
23,6542.0,19560.0,0.720479
24,6542.0,14316.0,0.984594


In [24]:
# Count NaN per cell
nan_mask = np.isnan(feats)
nan_per_cell = np.sum(nan_mask, axis=1)
meta['NaN_per_cell'] = nan_per_cell
meta

,Metadata_allele,Metadata_Plate,n_pos_pairs,n_total_pairs,average_precision,NaN_per_cell
0,FBP1_,2023-05-26_B1A1R1_P2T1,35,229,0.209349,0
1,FBP1_,2023-05-26_B1A1R1_P2T1,35,205,0.198680,0
2,FBP1_,2023-05-26_B1A1R1_P2T1,35,189,0.314503,0
3,GFAP_Glu205Lys,2023-05-26_B1A1R1_P2T1,15,180,0.089638,0
4,GFAP_Glu205Lys,2023-05-26_B1A1R1_P2T1,15,158,0.123770,0
...,...,...,...,...,...,...
26064,ZMYND10_Arg340Gln,2023-05-30_B1A1R1_P4T4,15,150,1.000000,9
26065,ZMYND10_Arg340Gln,2023-05-30_B1A1R1_P4T4,15,140,0.245668,0
26066,ZMYND10_Arg340Gln,2023-05-30_B1A1R1_P4T4,15,141,0.221692,0
26067,UBQLN2_Pro506Thr,2023-05-30_B1A1R1_P4T4,11,122,1.000000,1


Every cell that has even a single NaN feature gets a pairwise cosine similarity of NaN with each profile that it is compared to (see two examples in the table printed above). The strategy for handling ties in copairs return an average precision of 1 when the entire rank list of pairwise similarities are NaNs.